In [1]:
from logistic_agent import LogisticAgent

In [7]:
log_agent =  LogisticAgent(name="LogisticAgent")

[autogen.oai.client: 03-04 20:27:11] {261} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [2]:
# Initialize the agent
agent = LogisticAgent()

# Process a supply request
request = {
    "request_type": "get_supplies",
    "parameters": {
        "location": "Mountain Rescue Zone",
        "team": "6 members",
        "duration": 3,
        "resources": ["first aid kit", "thermal blankets", "climbing gear"]
    }
}

# Get supply recommendations
response = agent.process_request(request)

# Track resource usage after the mission
agent.track_resource_usage(
    mission_id="mountain_rescue_2024", 
    resources_used={
        "first aid kit": 1,
        "thermal blankets": 6,
        "climbing gear": 3
    }
)

# Retrieve recent incident logs
recent_incidents = agent.get_historical_incidents()

[autogen.oai.client: 03-04 20:28:57] {261} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [3]:
response

{'original_request': {'timestamp': '2025-03-04 20:28:57.160912',
  'full_message': {'request_type': 'get_supplies',
   'parameters': {'location': 'Mountain Rescue Zone',
    'team': '6 members',
    'duration': 3,
    'resources': ['first aid kit', 'thermal blankets', 'climbing gear']}}},
 'status': 'success',
 'recommendations': "Okay, let's craft a comprehensive supply recommendation for a 3-day mountain rescue mission with a team of 6, factoring in the given information and a blank-slate historical context.\n\n**1. Essential Supplies List with Quantities:**\n\n| Item                     | Quantity | Priority | Rationale                                                                                                                                                              |\n| ------------------------ | -------- | -------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------

In [4]:
import unittest
import json
import os
import datetime
from unittest.mock import Mock, patch
from typing import Dict, Any

# Import the LogisticAgent class (assuming it's in the same directory)
from logistic_agent import LogisticAgent

class TestLogisticAgent(unittest.TestCase):
    def setUp(self):
        """Initialize the LogisticAgent before each test"""
        self.agent = LogisticAgent()

    def test_process_request_get_supplies(self):
        """Test processing a get_supplies request"""
        request = {
            "request_type": "get_supplies",
            "parameters": {
                "location": "Mountain Rescue Zone",
                "team": "6 members",
                "duration": 3,
                "resources": ["first aid kit", "thermal blankets", "climbing gear"]
            }
        }
        
        response = self.agent.process_request(request)
        
        # Assertions
        self.assertEqual(response['status'], 'success')
        self.assertIn('recommendations', response)
        self.assertIn('metadata', response)
        self.assertEqual(response['metadata']['location'], 'Mountain Rescue Zone')

    def test_process_request_unknown_type(self):
        """Test processing an unknown request type"""
        request = {
            "request_type": "unknown_request",
            "parameters": {}
        }
        
        response = self.agent.process_request(request)
        
        # Assertions
        self.assertEqual(response['status'], 'error')
        self.assertIn('Unknown request type', response['message'])

    def test_log_incident(self):
        """Test logging an incident"""
        incident_data = {
            "status": "test",
            "message": "Test incident logging"
        }
        
        # Log the incident
        result = self.agent.log_incident(incident_data)
        
        # Assertions
        self.assertTrue(result)
        self.assertTrue(len(self.agent.historical_incidents) > 0)
        
        # Check the last logged incident
        last_incident = self.agent.historical_incidents[-1]
        self.assertEqual(last_incident['status'], 'test')
        self.assertEqual(last_incident['message'], 'Test incident logging')
        self.assertIn('timestamp', last_incident)

    def test_track_resource_usage(self):
        """Test tracking resource usage"""
        # First, add some initial resources
        self.agent.resources = {
            "first aid kit": 10,
            "thermal blankets": 20,
            "climbing gear": 5
        }
        
        # Track resource usage
        usage_log = self.agent.track_resource_usage(
            mission_id="test_mission",
            resources_used={
                "first aid kit": 2,
                "thermal blankets": 5
            }
        )
        
        # Assertions
        self.assertIn('mission_id', usage_log)
        self.assertIn('timestamp', usage_log)
        self.assertIn('resources_used', usage_log)
        
        # Check updated resource inventory
        self.assertEqual(self.agent.resources['first aid kit'], 8)
        self.assertEqual(self.agent.resources['thermal blankets'], 15)

    def test_get_historical_incidents(self):
        """Test retrieving historical incidents"""
        # Log multiple incidents
        for i in range(15):
            self.agent.log_incident({
                "status": f"test_incident_{i}",
                "message": f"Test incident {i}"
            })
        
        # Retrieve incidents
        recent_incidents = self.agent.get_historical_incidents(limit=10)
        
        # Assertions
        self.assertEqual(len(recent_incidents), 10)
        self.assertTrue(all('timestamp' in incident for incident in recent_incidents))

    def test_analyze_historical_incidents(self):
        """Test analyzing historical incidents for a location"""
        # Log incidents for different locations
        incidents = [
            {"location": "Mountain Rescue Zone", "message": "Cold conditions encountered"},
            {"location": "Mountain Rescue Zone", "message": "Difficult terrain navigation"},
            {"location": "Desert Rescue Zone", "message": "Extreme heat challenge"}
        ]
        
        for incident in incidents:
            self.agent.log_incident(incident)
        
        # Analyze incidents for Mountain Rescue Zone
        analysis = self.agent.analyze_historical_incidents("Mountain Rescue Zone")
        
        # Assertions
        self.assertIsInstance(analysis, str)
        self.assertIn("Mountain Rescue Zone", analysis)
        self.assertIn("Cold conditions encountered", analysis)
        self.assertIn("Difficult terrain navigation", analysis)

    @patch('logistic_agent.genai.Client')
    def test_generate_method(self, mock_genai_client):
        """Test the generate method with mocked Gemini client"""
        # Setup mock response
        mock_response = Mock()
        mock_response.text = "Mocked supply recommendation"
        
        mock_client_instance = Mock()
        mock_client_instance.models.generate_content.return_value = mock_response
        mock_genai_client.return_value = mock_client_instance
        
        # Call generate method
        prompt = "Generate supply recommendations for mountain rescue"
        response = self.agent.generate(prompt)
        
        # Assertions
        self.assertEqual(response, "Mocked supply recommendation")
        mock_client_instance.models.generate_content.assert_called_once()

    def test_assess_supply_needs(self):
        """Test assessing supply needs"""
        request = {
            "request_type": "assess_supply_needs",
            "parameters": {
                "location": "Tropical Forest",
                "conditions": {
                    "temperature": "35°C",
                    "humidity": "85%",
                    "terrain": "dense vegetation"
                }
            }
        }
        
        response = self.agent.process_request(request)
        
        # Assertions
        self.assertEqual(response['status'], 'success')
        self.assertIn('assessment', response)
        self.assertIn('metadata', response)
        self.assertEqual(response['metadata']['location'], 'Tropical Forest')

def run_tests():
    """Run the test suite and print results"""
    test_suite = unittest.TestLoader().loadTestsFromTestCase(TestLogisticAgent)
    test_runner = unittest.TextTestRunner(verbosity=2)
    result = test_runner.run(test_suite)
    
    # Optional: Return True if all tests pass, False otherwise
    return result.wasSuccessful()

if __name__ == '__main__':
    run_tests()

test_analyze_historical_incidents (__main__.TestLogisticAgent)
Test analyzing historical incidents for a location ... 

[autogen.oai.client: 03-04 20:30:46] {261} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


FAIL
test_assess_supply_needs (__main__.TestLogisticAgent)
Test assessing supply needs ... 

[autogen.oai.client: 03-04 20:30:46] {261} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


FAIL
test_generate_method (__main__.TestLogisticAgent)
Test the generate method with mocked Gemini client ... 

...

ok
test_process_request_unknown_type (__main__.TestLogisticAgent)
Test processing an unknown request type ... 

[autogen.oai.client: 03-04 20:31:33] {261} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


ok
test_track_resource_usage (__main__.TestLogisticAgent)
Test tracking resource usage ... 

[autogen.oai.client: 03-04 20:31:33] {261} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


ok

FAIL: test_analyze_historical_incidents (__main__.TestLogisticAgent)
Test analyzing historical incidents for a location
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/tk/hv2t5jr97w10vv7pz_qm8wfc0000gn/T/ipykernel_54810/3620701850.py", line 129, in test_analyze_historical_incidents
    self.assertIn("Mountain Rescue Zone", analysis)
AssertionError: 'Mountain Rescue Zone' not found in 'Previous Incident Insights:\n- 2025-03-04 20:30:46.511124: Cold conditions encountered\n- 2025-03-04 20:30:46.511149: Difficult terrain navigation\n'

FAIL: test_assess_supply_needs (__main__.TestLogisticAgent)
Test assessing supply needs
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/tk/hv2t5jr97w10vv7pz_qm8wfc0000gn/T/ipykernel_54810/3620701850.py", line 169, in test_assess_supply_needs
    self.assertEqual(response['status'], 'success')
Asser